<a href="https://colab.research.google.com/github/DigiClau/stablediffusion_webui/blob/main/StableDiffusionWebUI_digiclau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 이 Google Colab notebook은 [AUTOMATIC1111 Stable Diffusion Web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)를 실행해 줍니다.

### [Stable Diffusion Google Colab 사용 설명 영상](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw) - 자세한 사용 설명을 보실수 있습니다.

### [DigiClau Youtube](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw) - 저의 채널도 방문해 주세요.

###이 Google Colab은 필요에 따라 업데이트 됩니다. 최신 버전은 [이곳](https://www.youtube.com/channel/UCllY989CI5F-eU1BJ-rwEvw)에 있습니다.

## 저 커피한잔 사주세요

## Instruction(필수 사항)
In the next cell...
1. 사용할 모델들을 선택합니다.
2. 외쪽에 있는 Play버튼을 눌러서 시작합니다.
3. 모든 설치가 끝날때까지 기다립니다. 시간이 걸립니다.
4. 설치가 끝난후, public링크를 누르면, 새로운 웹브라우저 창에서 Stable Diffusion 이 시작됩니다.
5. 더 자세한 내용은, 위에 있는 사용 설명 영상을 보시기 바랍니다.

## ngrok (선택 사항)
AUTOMATIC1111은 Gradio 라이브러리를 사용해서, Web UI를 제공합니다. 하지만, Google Colab에서 사용시, 접속이 끊기거나, 반응하지 않는 경우가 종종(혹은 자주) 발생합니다.\
[ngrok](https://ngrok.com/)을 사용하면 그러한 문제를 방지할 수 있습니다.  Google Colab에서 Stable Diffusion을 사용하시다가, WebUI 사용에 문제가 발생한다면 [ngrok](https://ngrok.com/)을 사용하시기 바랍니다.\
ngrok 사용방법:
  1. ngrok 웹사이트로 갑니다. https://ngrok.com/
  2. ngrok 계좌를 만듭니다. 무료입니다.
  3. ngrok 웹사이트에서 Authtoken을 만듭니다.
  4. Authtoken을 복사해서, 아래에 있는 ngrok란에 붙여줍니다.

## 업데이트
- 2023년 7월 8일: AUTOMATIC1111 1.4.0 버전


In [ ]:
import os, subprocess, time, glob
#@title <font size="6" color="black">Stable Diffusion 시작하는 섹션</font>
output_path = 'digiclau_lab'
#@markdown ## (선택사항) ngrok Authtoken을 이곳에 넣으면, ngrok GUI가 사용됩니다..
#@markdown ### 이곳을 공백으로 남겨두면, Gradio GUI가 기본으로 사용됩니다.
ngrok ='' #@param {type: "string"}

#@markdown ## 구글 드라이브에 연결할지를 결정합니다.
#@markdown ### Yes를 선택하면, 구글 드라이브에 'digiclau_lab'이라는 폴더가 생기고, 그 안에 세팅 파일과, 생성된 이미지들이 저장됩니다.
#@markdown ### No를 선택하면, 구글 드라이브는 필요없지만, 세팅설정과 생성된 이미지들은 저장되지 않습니다.
use_gdrive = 'yes' #@param ['yes', 'no']
class GDriveSaveMode:
#    Everything = 1
    ON = 1
    OFF = 0
if use_gdrive == 'yes':
  is_drive_used = True
elif use_gdrive == 'no':
  is_drive_used = False
#elif use_gdrive == 'Everything':
#  is_drive_used = GDriveSaveMode.Everything
else:
  raise ValueError("use_gdrive value invalid.")

#@markdown ## 아래의 Checkpoint 모델중에, 사용할 모델을 선택하세요.
#@markdown ### Stable Diffusion 1.5 모델은 기본으로 설치됩니다.
v1_5_model = True
F222_model = False #@param{type: "boolean"}
DreamShaper_model = False #@param{type: "boolean"}
OpenJourney_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}
Inkpunk_Diffusion_model = False #@param{type: "boolean"}
instruct_pix2pix_model = False #@param{type: "boolean"}
v2_1_768_model = False #@param{type: "boolean"}
v2_1_512_model = False #@param{type: "boolean"}
v2_depth_model = False #@param{type: "boolean"}

#@markdown ## 아래의 확장 프로그램(Extensions)중에, 사용할 모델을 선택하세요.
ControlNet = False #@param{type: "boolean"}
Deforum = False #@param{type: "boolean"}
Regional_Prompter = False #@param{type: "boolean"}
Ultimate_SD_Upscale = False #@param{type: "boolean"}
Openpose_Editor = False #@param{type: "boolean"}

#@markdown ## 로그 삭제
#Clear_Log = True #@param{type:'boolean'}

#@markdown ## 위에 있는 Checkpoint 모델 외에 사용하고 싶은 모델의 다운로드 링크를 넣어주세요.
#@markdown ### 각각의 링크 주소는 쉼표(,)로 구분해주세요.
Model_from_URL = '' #@param {type: "string"}
Save_a_copy_in_Google_Drive = False #@param{type: "boolean"}
#@markdown (Saved models will be available next time you start)


#@markdown ## 위에 있는 확장 프로그램(Extensions) 외에 사용하고 싶은 모델의 URL을 넣어주세요.
#@markdown ### 각각의 URL 주소는 쉼표(,)로 구분해주세요.
Extensions_from_URL = 'https://github.com/vladmandic/sd-extension-steps-animation' #@param {type: "string"}

#@markdown ## COMMANDLINE_ARGS= 에 추가할 값이 있다면 이곳에 넣어주세요.
Extra_arguments = '' #@param {type: "string"}


if is_drive_used # != GDriveSaveMode.OFF:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive')
  output_path = '/content/drive/MyDrive/' + output_path
#  if is_drive_used == GDriveSaveMode.Everything:
#    # install A1111 in AI_PIC
#    root = output_path
#  elif is_drive_used == GDriveSaveMode.ON: # 불필요
    # install A1111 in Colab
  root = '/content/'
    # make necessary directories if not exists
  !mkdir -p {output_path}/outputs
  !mkdir -p {output_path}/models
  !mkdir -p {output_path}/ESRGAN
  !mkdir -p {output_path}/hypernetworks
#  else:
#    raise ValueError("Unexpected is_drive_used: %s"%is_drive_used)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/'

def clear():
    from IPython.display import clear_output; return clear_output()

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('Downloading models ...')
  if v1_5_model:
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')
  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix_half.ckpt')
  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_1_512_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_512-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml

  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and is_drive_used: # == GDriveSaveMode.ON:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
#    print("Create new ui-config.json file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
#    print("Create new config.json file.")
    !wget https://raw.githubusercontent.com/DigiClau/stablediffusion_webui/main/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs

  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE


def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt

  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')

  print("ControlNet install completed.")

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {root}/stable-diffusion-webui/extensions/deforum
  #!cd {root}/stable-diffusion-webui/extensions/deforum; git checkout c42834645805e0f26172888b29f5a9210063db14

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter {root}/stable-diffusion-webui/extensions/sd-webui-regional-prompter


def installUltimateSDUpscale():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111

def installOpenPoseEditor():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/fkunn1326/openpose-editor

def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    !git clone {url}

updatePython()
!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# A1111 first launch
%cd {root}/stable-diffusion-webui
!git checkout -f 394ffa7b0a7fff3ec484bcd084e673a8b301ccc8
!COMMANDLINE_ARGS="--exit"  python launch.py

if is_drive_used: # == GDriveSaveMode.ON:
  initSaveGoogleDriveModelOnly()

download_models()

if ControlNet:
  installControlNet()

if Deforum:
  installDeforum()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()

if Openpose_Editor:
  installOpenPoseEditor()

installExtensionsFromURL(Extensions_from_URL)

#if Clear_Log:
clear()

%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'

if ngrok:
  args += f' --ngrok {ngrok} '
else:
  args += ' --share '
args+= ' '+Extra_arguments
print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}

In [ ]:
#@title # Download checkpoint models

#@markdown ### Install models from URL [huggingface.co or civitai.com] (separate with comma)
models_list = '' #@param {type: "string"}

def downloadModel(url):
    if 'huggingface.co' in url:
        filename = url.split('/')[-1]
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
    else:
        # civitai
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

if models_list:
    %cd /content/drive/MyDrive/Claue_SD/stable-diffusion-webui/models/Stable-diffusion
    for model_URL in models_list.split(','):
        downloadModel(model_URL)